# Clasificación MNIST con red convolucional

MNIST es un ejemplo clásico de reconocimiento de dígitos escritos a mano. Se utiliza la base de datos MNIST que contiene 60,000 imágenes de entrenamiento y 10,000 imágenes de prueba. Cada imagen es de 28x28 pixeles y cada pixel tiene un valor entre 0 y 255.

Es el mismo ejemplo con el que se explica la [teoría sobre redes neuronales en el video de 3Brown1Blue](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) ([versión doblada a español](https://www.youtube.com/watch?v=jKCQsndqEGQ)).

In [6]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import datasets, transforms

## Carga del dataset

A menudo usaremos más de una transformación para preprocesar los datos. Por ejemplo, en el caso de las imágenes, a menudo se normalizan y se redimensionan. Para hacer esto de manera eficiente, podemos usar la clase `Compose` de `torchvision.transforms`. 

In [7]:
# Definimos las transformaciones para preprocesado de las imágenes
transform = transforms.Compose([transforms.ToTensor(),])

# Cargamos el dataset MNIST
train_data = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST('./data', train=False, download=True, transform=transform)

# Creamos los dataloaders
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

## Definición del modelo

In [8]:
from torch.nn import functional as F

class CNN(nn.Module): # Definimos la red neuronal convolucional
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1) 
    self.pool = nn.MaxPool2d(2, 2)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.fc1 = nn.Linear(7 * 7 * 64, 128)
    self.fc2 = nn.Linear(128, 10)

  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1, 7 * 7 * 64) # Aplanamiento tras las capas convolucionales
    x = F.relu(self.fc1(x))
    x = F.log_softmax(self.fc2(x), dim=1)  # Use log_softmax for cross-entropy loss
    return x
  
model = CNN() # Instanciamos la red neuronal

## Entrenamiento del modelo

### Definición de la función de pérdida y el optimizador

Definimos la función de perdida y el optimizador. En este caso usaremos el optimizador `optim.Adam`. Adam es una variante del descenso de gradiente estocástico que calcula tasas de aprendizaje individuales para diferentes parámetros.

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

### Entrenamiento

In [10]:
model.train() # Ponemos el modelo en modo entrenamiento (no es necesario por ser el comportamiento por defecto, pero es una buena práctica)

for epoch in range(5): # Definimos 10 epochs
  
  for i, (images, labels) in enumerate(train_loader):
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{10}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/938], Loss: 0.1327
Epoch [1/10], Step [200/938], Loss: 0.1014
Epoch [1/10], Step [300/938], Loss: 0.2550
Epoch [1/10], Step [400/938], Loss: 0.1968
Epoch [1/10], Step [500/938], Loss: 0.0691
Epoch [1/10], Step [600/938], Loss: 0.0590
Epoch [1/10], Step [700/938], Loss: 0.0292
Epoch [1/10], Step [800/938], Loss: 0.0677
Epoch [1/10], Step [900/938], Loss: 0.0063
Epoch [2/10], Step [100/938], Loss: 0.0157
Epoch [2/10], Step [200/938], Loss: 0.0326
Epoch [2/10], Step [300/938], Loss: 0.0450
Epoch [2/10], Step [400/938], Loss: 0.0234
Epoch [2/10], Step [500/938], Loss: 0.0143
Epoch [2/10], Step [600/938], Loss: 0.0302
Epoch [2/10], Step [700/938], Loss: 0.0092
Epoch [2/10], Step [800/938], Loss: 0.0081
Epoch [2/10], Step [900/938], Loss: 0.0205
Epoch [3/10], Step [100/938], Loss: 0.0077
Epoch [3/10], Step [200/938], Loss: 0.0703
Epoch [3/10], Step [300/938], Loss: 0.0136
Epoch [3/10], Step [400/938], Loss: 0.0555
Epoch [3/10], Step [500/938], Loss: 0.0825
Epoch [3/10

## Evaluación del modelo

In [11]:
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
  print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 99.01%


- https://dudeperf3ct.github.io/cnn/mnist/2018/10/17/Force-of-Convolutional-Neural-Networks/